#leotta_2021_load_dataset.ipynb
This data set loader uses the leotta_2021_get_X_y_sub.py file generated by downloading the python version of the same name Jupyter notebook.

It will perform a train/test (and optional validation) split and one-hot encode the activity labels.   Returns x/y_train and x/y_test numpy arrays that may be fed directly into a neural net model.

Example usage:

    x_train, y_train, x_test, y_test = leotta_2021_load_dataset()
  

Developed and tested using colab.research.google.com
IMPORTANT a high RAM runtime is required. Select runtime > change type > shape = high RAM  
To save as .py version use File > Download .py

Author:  [Lee B. Hinkle](https://userweb.cs.txstate.edu/~lbh31/), [IMICS Lab](https://imics.wp.txstate.edu/), Texas State University, 2023

<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike 4.0 International License</a>.

TODOs:
* Integration of get_X_y_sub code and conversion to the more standard format in progress.


In [13]:
import os
import shutil #https://docs.python.org/3/library/shutil.html
from shutil import unpack_archive # to unzip
import urllib.request # to get files from web w/o !wget
import requests #for downloading zip file
import pandas as pd
import numpy as np
# from tabulate import tabulate # for verbose tables, showing data
import matplotlib.pyplot as plt
# from tensorflow.keras.utils import to_categorical # for one-hot encoding
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder
# from sklearn.preprocessing import OneHotEncoder

# Load shared transform (xforms) functions and utils from IMICS Public Repo



In [2]:
def get_web_file(fname, url):
    """checks for local file, if none downloads from URL.    
    :return: nothing"""
    if (os.path.exists(fname)):
        print ("Local",fname, "found, skipping download")
    else:
        print("Downloading",fname, "from", url)
        urllib.request.urlretrieve(url, filename=fname)

In [3]:
try:
    import load_data_transforms as xforms
except:
    get_web_file(fname = 'load_data_transforms.py', url = 'https://raw.githubusercontent.com/imics-lab/load_data_time_series/main/load_data_transforms.py')
    import load_data_transforms as xforms

try:
    import load_data_utils as utils  
except:
    get_web_file(fname = 'load_data_utils.py', url = 'https://raw.githubusercontent.com/imics-lab/load_data_time_series/main/load_data_utils.py')
    import load_data_utils as utils

# Global and Dataset Parameters

In [4]:
# environment and execution parameters
my_dir = '.' # replace with absolute path if desired
dataset_dir = os.path.join(my_dir,'dataset') # temp dir for processing

interactive = True # for exploring data and functions interactively
verbose = True

log_info = "" # a global to append dataset processing info

# dataset parameters
all_channel_list = ['accel_x', 'accel_y', 'accel_z','accel_ttl','bvp','eda','p_temp']
# frequency = 32 - unlike some of the other loaders this is hardcoded due to
# the unique sample freqencies that differ between the individual e4 sensors
xforms.time_steps = 96 # three seconds at 32Hz
xforms.stride = 32 # one second step for each sliding window
# The label_map_<dataset> contains a mapping from strings to ints for all
# possible labels in the entire dataset.   This allows for predictable conversion
# regardless of the slices.
subj_alloc_dict = dict (train_subj = [1,2,7,8], valid_subj = [3,6], test_subj = [4,5])
label_map_leotta = {"label":     {'OTHER':0,'RELAX':1,'KEYBOARD_WRITING':2,
                                  'LAPTOP':3,'HANDWRITING':4,'HANDWASHING':5,
                                  'FACEWASHING':6,'TEETHBRUSH':7,'SWEEPING':8,
                                  'VACUUMING':9,'EATING':10,'DUSTING':11,
                                  'RUBBING':12,'DOWNSTAIRS':13,'WALKING':14,
                                  'WALKING_FAST':15,'UPSTAIRS_FAST':16,
                                  'UPSTAIRS':17}} # from README.txt

In [5]:
interactive = False # runs when saved as .py, skip cell if developing/debugging

In [6]:
# Please go to https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/G23QTS
# to read through the terms and also find the proper citations if you use this dataset.
# This is also where you can download directly if the link below fails.
!gdown "1P5PIYeYvbfL4kQj-P2sm-JquUIddAxec&confirm=t" # ADL_Leotta_2021.zip

Downloading...
From: https://drive.google.com/uc?id=1P5PIYeYvbfL4kQj-P2sm-JquUIddAxec&confirm=t
To: /content/ADL_Leotta_2021.zip
100% 258M/258M [00:01<00:00, 135MB/s]


The !gdown above works in colab but not as .py.   Need to see if an alternate method is possible.

In [ ]:
# import gdown
# url = 'https://drive.google.com/file/d/1P5PIYeYvbfL4kQj-P2sm-JquUIddAxec/view?usp=share_link' 
# out_path = 'ADL_Leotta_2021.zip'
# gdown.download(url, out_path, quiet=False)

In [7]:
def unzip_leotta():
    """check for local copy, if none unzips the dataset structure in working_dir"""
    if (os.path.isdir(dataset_dir)):
        print("Using existing Leotta archive in", dataset_dir)
        return
    else:
        print("Unzipping Leotta 2021 dataset into", dataset_dir)
        zip_ffname = os.path.join(my_dir, 'ADL_Leotta_2021.zip')
        if (os.path.exists(zip_ffname)):
            print("Using source file", zip_ffname)
            shutil.unpack_archive(zip_ffname,dataset_dir,'zip')
        else:
            print("ERROR: ", zip_ffname, " not found")
            print("Go to https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/G23QTS")
            print("Click access dataset on right, accept terms, download zip and place in current directory")
            print("with filename ADL_Leotta_2021.zip, should be a 246.2MB zip file")
            return
if interactive:
    unzip_leotta()

In [8]:
def df_from_csv (
    sub_num, # 1 - 8
    sensor_loc): # ankle, hip, wrist
    """reads Leotta_2021 csv, returns df with accel x/y/z/ttl, label, sub_num
    args:
        sub_num (int) subject number from 1 to 8
        sensor_loc (string) sensor location ankle, hip, or wrist
    returns:
        An IR1 format dataframe, note labels are int encoded as in the raw dataset"""
    fnameX = sensor_loc + '_X_0' + str(sub_num) +  '.csv'
    fnamey = sensor_loc + '_Y_0' + str(sub_num) +  '.csv'
    ffnameX = os.path.join(dataset_dir, sensor_loc, fnameX)
    ffnamey = os.path.join(dataset_dir, sensor_loc, fnamey)
    if verbose:
        print ('df_from_csv processing: ', ffnameX, ffnamey)
    df = pd.read_csv(ffnameX)
    if (sensor_loc == 'wrist'): # Centrepoint device has different header name
        df.rename(columns={'Timestamp UTC': 'Timestamp'}, inplace=True)
    # the imported Timestamp is an object - need to convert to DateTime
    # in order to set the index to DateTime format.  Enables resampling etc.
    # Leaving these here - helpful to debug if leveraging this code!
        #print("*** Start ***")
        #print(type(df.index))
        #print(df.info(verbose=True))  
    df['Timestamp'] = pd.to_datetime(df['Timestamp']) 
    df.set_index('Timestamp', drop = True, inplace = True)
    if (sensor_loc != 'wrist'): # Centrepoint doesn't have non-accel columnns
        df = df.drop(['Temperature','Gyroscope X','Gyroscope Y','Gyroscope Z',
                      'Magnetometer X','Magnetometer Y','Magnetometer Z'], axis=1)
    df_sqd = df.pow(2)[['Accelerometer X','Accelerometer Y','Accelerometer Z']] #square each accel
    df_sum = df_sqd.sum(axis=1) #add sum of squares, new 1 col df
    df.loc[:,'accel_ttl'] = df_sum.pow(0.5)-1  # sqrt and remove 1g due to gravity
    del df_sqd, df_sum
    df.columns = [sensor_loc + '_accel_x', sensor_loc + '_accel_y', sensor_loc + '_accel_z', sensor_loc + '_accel_ttl']
    # tighten up the column types for space savings, probably should be function in utils or xforms
    # change to 32-bit, credit/ref https://stackoverflow.com/questions/69188132/how-to-convert-all-float64-columns-to-float32-in-pandas
    # Select columns with 'float64' dtype  
    float64_cols = list(df.select_dtypes(include='float64'))
    # The same code again calling the columns
    df[float64_cols] = df[float64_cols].astype('float32')
    # add activity numbers - number of rows are the same in this dataset
    # Why doesn't this work? df['label'] = pd.read_csv(ffnamey, dtype='Int64')
    dfy = pd.read_csv(ffnamey)
    df['label']=dfy['label'].to_numpy() # this works, above doesn't?
    df['label'] = df['label'].astype(np.int8) # change from float to int
    del dfy
    # add column with subject number
    df['sub'] = np.int8(sub_num)
    return df
if interactive:
    snum = 1
    df_ankle = df_from_csv(sub_num = snum, sensor_loc = 'ankle')
    df_hip = df_from_csv(sub_num = snum, sensor_loc = 'hip')
    df_wrist = df_from_csv(sub_num = snum, sensor_loc = 'wrist')
    display(df_ankle.info())
    display(df_hip.info())
    display(df_wrist.info())
    display(df_ankle.head())
    display(df_hip.head())
    display(df_wrist.head())


In [9]:
def df_from_one_sub (sub_num): # 1 - 8
    """reads 3 csv files for a single subject, combines an returns a single dataframe"""
    my_sub_num = sub_num # not sure necessary but easier to follow...
    df_ankle = df_from_csv(sub_num = my_sub_num, sensor_loc = 'ankle')
    df_hip = df_from_csv(sub_num = my_sub_num, sensor_loc = 'hip')
    #wrist is a bit more complicated since the sample rate is different
    df_wrist = df_from_csv(sub_num = my_sub_num, sensor_loc = 'wrist')
    df_wrist = xforms.to_fixed_ir1_timedelta(df_wrist,new_time_step='10ms')

    if ((df_ankle['label'].equals(df_hip['label']))
            and (df_ankle['sub'].equals(df_hip['sub']))
            and (df_ankle['label'].equals(df_wrist['label']))
            and (df_ankle['sub'].equals(df_wrist['sub']))) :
        if verbose:
            print('confirmed label and sub match - dropping from ankle and hip')
        df_ankle.drop(['label','sub'], axis=1, inplace=True)
        df_hip.drop(['label','sub'], axis=1, inplace=True)
    else:
        print('Error:  label and sub do not match, cannot combine dataframes')
        print('label match = ',df_ankle['label'].equals(df_hip['label']))
        print('sub match = ',df_ankle['sub'].equals(df_hip['sub']))
    df_temp = df_ankle.join(df_hip)
    df_final = df_temp.join(df_wrist)
    del df_temp
    df_final = xforms.convert_ir1_labels_to_strings(df = df_final, label_map = label_map_leotta)
    return df_final
if interactive:
    df_temp = df_from_one_sub (sub_num = 8)
    print(type(df_temp.index)) # should be DateTimeIndex
    print(df_temp.info(verbose=True))
    display(df_temp.head()) 

In [10]:
def get_leotta_ir1_dict():
    """reads the Leotta dataset and converts each "session file" to an IR1
    dataframe.  The goal here is to capture and convert all raw data into
    a 2D dataframe of rows = datetime index of each sample, columns = {channels,
    label(s), subject_num}.  Additional methods may be used to drop channels,
    and convert the string labels to mapped ints prior to switch to ndarrays.
    Args:
        none 
    Returns: a dict containing key = df_name and item = IR1 dataframes."""
    unzip_leotta()
    ir1_df_dict = dict() # an empty dictionary
    for i in range(1,9):
        ir1_name = "Leotta_Sub" + str(i)
        print('Processing subject number', i, "into", ir1_name)
        df_temp = df_from_one_sub (sub_num = i)
        ir1_df_dict[ir1_name]=df_temp # key is root name in the file
    return ir1_df_dict
if interactive:
    verbose = False
    ir1_dict = get_leotta_ir1_dict()
    print('IR1 dataframes:',ir1_dict.keys())
    for df_name, df in ir1_dict.items():
        display(df.head())
        break # just want one
    verbose = True

# Start of original loader code

In [11]:
def leotta_2021_load_dataset(
    verbose = True,
    incl_xyz_accel = False, # include component accel_x/y/z in ____X data
    incl_rms_accel = True, # add rms value (total accel) of accel_x/y/z in ____X data
    incl_val_group = False, # split train into train and validate

    one_hot_encode = True # make y into multi-column one-hot, one for each activity
    ):
    """calls e4_get_X_y_sub and processes the returned arrays by separating
    into _train, _validate, and _test arrays for X and y based on split_sub
    dictionary."""
    orig_zipfile = '/content/drive/My Drive/Datasets/ADL_Leotta_2021.zip'
    X, y, sub, xys_info = get_X_y_sub(orig_zipfile=orig_zipfile)
    log_info = 'Processing'+str(orig_zipfile)
    #remove component accel if needed
    if (not incl_xyz_accel):
        print("Removing component accel")
        X = np.delete(X, [0,1,2,4,5,6,8,9,10], 2)
    if (not incl_rms_accel):
        print("Removing total accel")
        X = np.delete(X, [3,7,11], 2)  
    #One-Hot-Encode y...there must be a better way when starting with strings
    #https://machinelearningmastery.com/how-to-one-hot-encode-sequence-data-in-python/

    if (one_hot_encode):
        # integer encode
        y_vector = np.ravel(y) #encoder won't take column vector
        le = LabelEncoder()
        integer_encoded = le.fit_transform(y_vector) #convert from string to int
        name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
        print("One-hot-encoding: category names -> int -> one-hot")
        print(name_mapping) # seems risky as interim step before one-hot
        log_info += "One Hot:" + str(name_mapping) +"\n\n"
        onehot_encoder = OneHotEncoder(sparse=False)
        integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
        onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
        print("One-hot-encoding",onehot_encoder.categories_)
        y=onehot_encoded
        #return X,y
    # split by subject number pass in dictionary
    sub_num = np.ravel(sub[ : , 0] ) # convert shape to (1047,)
    if (not incl_val_group):
        train_index = np.nonzero(np.isin(sub_num, split_subj['train_subj'] + 
                                        split_subj['validation_subj']))
        x_train = X[train_index]
        y_train = y[train_index]
    else:
        train_index = np.nonzero(np.isin(sub_num, split_subj['train_subj']))
        x_train = X[train_index]
        y_train = y[train_index]

        validation_index = np.nonzero(np.isin(sub_num, split_subj['validation_subj']))
        x_validation = X[validation_index]
        y_validation = y[validation_index]

    test_index = np.nonzero(np.isin(sub_num, split_subj['test_subj']))
    x_test = X[test_index]
    y_test = y[test_index]
    if (incl_val_group):
        return x_train, y_train, x_validation, y_validation, x_test, y_test
    else:
        return x_train, y_train, x_test, y_test


        if(verbose):
            headers = ("Reshaped data","shape", "object type", "data type")
            mydata = [("x_train:", x_train.shape, type(x_train), x_train.dtype),
                    ("y_train:", y_train.shape ,type(y_train), y_train.dtype),
                    ("x_test:", x_test.shape, type(x_test), x_test.dtype),
                    ("y_test:", y_test.shape ,type(y_test), y_test.dtype)]
            print(tabulate(mydata, headers=headers))

        return x_train, y_train, x_test, y_test

In [12]:
if __name__ == "__main__":
    verbose = False
    ir1_dict = get_leotta_ir1_dict()
    print('IR1 dataframes:',ir1_dict.keys())
    for df_name, df in ir1_dict.items():
        print(df.head())
        break # just want one
    verbose = True
    # print("Downloading and processing Leotta 2021 dataset")
    # x_train, y_train, x_test, y_test = leotta_2021_load_dataset()
    # print("\nreturned arrays without validation group:")
    # print("x_train shape ",x_train.shape," y_train shape ", y_train.shape)
    # print("x_test shape  ",x_test.shape," y_test shape  ",y_test.shape)

    # x_train, y_train, x_validation, y_validation, x_test, y_test = leotta_2021_load_dataset(incl_val_group=True)
    # print("\nreturned arrays with validation group:")
    # print("x_train shape ",x_train.shape," y_train shape ", y_train.shape)
    # print("x_validation shape ",x_validation.shape," y_validation shape ", y_validation.shape)
    # print("x_test shape  ",x_test.shape," y_test shape  ",y_test.shape)

Unzipping Leotta 2021 dataset into ./dataset
Using source file ./ADL_Leotta_2021.zip
Processing subject number 1 into Leotta_Sub1
Processing subject number 2 into Leotta_Sub2
Processing subject number 3 into Leotta_Sub3
Processing subject number 4 into Leotta_Sub4
Processing subject number 5 into Leotta_Sub5
Processing subject number 6 into Leotta_Sub6
Processing subject number 7 into Leotta_Sub7
Processing subject number 8 into Leotta_Sub8
IR1 dataframes: dict_keys(['Leotta_Sub1', 'Leotta_Sub2', 'Leotta_Sub3', 'Leotta_Sub4', 'Leotta_Sub5', 'Leotta_Sub6', 'Leotta_Sub7', 'Leotta_Sub8'])


,ankle_accel_x,ankle_accel_y,ankle_accel_z,ankle_accel_ttl,hip_accel_x,hip_accel_y,hip_accel_z,hip_accel_ttl,wrist_accel_x,wrist_accel_y,wrist_accel_z,wrist_accel_ttl,label,sub
Timestamp,,,,,,,,,,,,,,
2020-07-30 11:21:00.000,-0.035156,-1.027832,0.029785,0.028864,-0.583984,-0.764160,0.374023,0.031926,0.475777,0.029808,0.934387,0.049075,OTHER,1
2020-07-30 11:21:00.010,-0.022949,-1.023926,0.033203,0.024721,-0.579102,-0.756348,0.374512,0.023563,0.481189,0.013713,0.918378,0.036949,OTHER,1
2020-07-30 11:21:00.020,-0.010254,-1.028320,0.038574,0.029094,-0.578613,-0.758301,0.367676,0.022252,0.490935,0.032158,0.904531,0.029686,OTHER,1
2020-07-30 11:21:00.030,-0.000977,-1.026855,0.048340,0.027993,-0.578125,-0.756836,0.374023,0.023192,0.498793,0.031644,0.909606,0.037908,OTHER,1
2020-07-30 11:21:00.040,0.003906,-1.031250,0.038574,0.031979,-0.575195,-0.755371,0.372559,0.019919,0.514537,0.030619,0.906663,0.042950,OTHER,1
